In [26]:
import numpy as np
import gym
import math
import random
from collections import namedtuple
from itertools import count
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
from torch.autograd import Variable

In [27]:
env = gym.make('CartPole-v1')
env = gym.make('Amidar-ram-v0')

STATES = 128

In [28]:
Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))

class ReplayMemory:
    
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = list()
        self.position = 0
    
    def push(self, *args):
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        
        self.position += 1
        if self.position == self.capacity:
            self.position = 0
            
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)
        

In [29]:
class DQN(nn.Module):

    def __init__(self, outputs):
        super(DQN, self).__init__()
        
        self.val_1 = nn.Linear(STATES, 320)
        self.val_2 = nn.Linear(320, 1)
        
        self.adv_1 = nn.Linear(STATES, 320)
        self.adv_2 = nn.Linear(320, outputs)

    def forward(self, x):
        val = F.relu(self.val_1(x))
        adv = F.relu(self.adv_1(x))
        
        val = self.val_2(val)
        adv = self.adv_2(adv)
        
        batch_size = adv.size(0)
        outputs = adv.size(1)
 
        adv = adv - adv.mean(1).unsqueeze(1).expand(batch_size, outputs)
        val = val.expand(batch_size, outputs) 
        x = val + adv
        return x

In [30]:
BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
MEMORY_SIZE = 10000
TARGET_UPDATE = 10

In [31]:
n_actions = env.action_space.n
steps_done = 0
episode_durations = list()

net = DQN(n_actions)
target_net = DQN(n_actions)
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
memory = ReplayMemory(MEMORY_SIZE)

In [32]:
def plot_durations():
    plt.figure(2)
    plt.clf()
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated

In [33]:
def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            return net(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[random.randrange(n_actions)]], dtype=torch.long)

In [34]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)

    batch = Transition(*zip(*transitions))
    
    batch_state = Variable(torch.cat(batch.state))
    batch_action = Variable(torch.cat(batch.action))
    batch_reward = Variable(torch.cat(batch.reward))
    batch_next_state = Variable(torch.cat(batch.next_state))

    state_action_values = net(batch_state).gather(1, batch_action)
    next_action = net(batch_next_state).max(1)[1].unsqueeze(1).detach()
    next_values = target_net(batch_next_state).gather(1, next_action).squeeze(1).detach()
    
    expected_state_action_values = (next_values * GAMMA) + batch_reward

    loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
num_episodes = [50] * 2 + [5] * 1
for epoch in range(len(num_episodes)):
    for i_episode in range(num_episodes[epoch]):
        state = torch.FloatTensor([env.reset()])
        for t in count():
            action = select_action(state)
            if epoch == len(num_episodes) - 1:
                env.render()
            next_state, reward, done, _ = env.step(action.item())
            next_state = torch.FloatTensor([next_state])
#             if done and t + 1 != 500:
#                 reward = -1
            reward = torch.FloatTensor([reward])

            memory.push(state, action, next_state, reward)
            state = next_state
            optimize_model()

            if done:
                episode_durations.append(t + 1)
                break
        if i_episode % TARGET_UPDATE == 0:
            target_net.load_state_dict(net.state_dict())

    print('Complete epoch {}'.format(epoch))
    print(np.mean(episode_durations[-50:]))

env.close()
plot_durations()
plt.show()

Complete epoch 0
777.98
